In [379]:
import pandas as pd

def split_str(str):
    word = ""
    for char in str:
        if not char.isdigit():
            word += char
    return word

def remove_symb(str):
    word = ""
    symbol = "("
    for char in str:
        if not char == symbol:
            word += char
        else:
            break
    word = word.strip()
    return word

def answer_one():
    file = "Energy Indicators.xls"
    energy = pd.read_excel(file,skiprows= 17, nrows= 227 )
    del energy['Unnamed: 0']
    del energy['Unnamed: 1']
    energy.rename( columns= {'Unnamed: 2':'Country',
                         "Petajoules": 'Energy Supply',
                         "Gigajoules": 'Energy Supply per Capita', "%":'% Renewable'}, inplace = True) 
    energy["Country"] = energy["Country"].apply(split_str)
    energy["Country"] = energy["Country"].apply(remove_symb)
    energy["Energy Supply"] = energy["Energy Supply"] * 1000000
    energy["Energy Supply"] = energy["Energy Supply"].apply(lambda x: x if type(x) is int else None)
    energy["Country"] = energy["Country"].apply(lambda x: "South Korea" if x == "Republic of Korea" else x)
    energy["Country"] = energy["Country"].apply(lambda x: "United States" if x == "United States of America" else x)
    energy["Country"] = energy["Country"].apply(lambda x: "United Kingdom" if x == "United Kingdom of Great Britain and Northern Ireland" else x)
    energy["Country"] = energy["Country"].apply(lambda x: "Hong Kong" if x == "China, Hong Kong Special Administrative Region" else x)
    
    file_gdp = "world_bank.csv"
    GDP = pd.read_csv(file_gdp, sep = ',',
                     dtype = None, skiprows=4, encoding='utf-8')

    GDP.rename( columns= {'Country Name':'Country'}, inplace = True) 

    GDP["Country"] = GDP["Country"].apply(lambda x: "South Korea" if x == "Korea, Rep." else x)
    GDP["Country"] = GDP["Country"].apply(lambda x: "Iran" if x == "Iran, Islamic Rep." else x)
    GDP["Country"] = GDP["Country"].apply(lambda x: "Hong Kong" if x == "Hong Kong SAR, China" else x)
    gdp = GDP[["Country", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015"]]
    
    ScimEn_file = "country_rank.xlsx"
    ScimEn = pd.read_excel(ScimEn_file)
    del ScimEn["Region"]
    ScimEn_rows = ScimEn.iloc[0:15]
    merge_1 = ScimEn_rows.merge(energy, how="inner", on = 'Country')
    result = merge_1.merge(gdp, how="left", on = 'Country')
    result = result.set_index(['Country'])
    return result

answer_one()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,303064,301778,3036531,2092737,10.02,273,1.271910e+11,93,19.754910,2.752132e+12,3.550343e+12,4.594307e+12,5.101703e+12,6.087164e+12,7.551500e+12,8.532230e+12,9.570406e+12,1.047568e+13,1.106155e+13
United States,2,184851,181106,2623922,819242,14.19,389,9.083800e+10,286,11.570980,1.381559e+13,1.447423e+13,1.476986e+13,1.447806e+13,1.504896e+13,1.559973e+13,1.625397e+13,1.684319e+13,1.755068e+13,1.820602e+13
India,3,60257,58589,590570,213760,9.80,198,3.319500e+10,26,14.969080,9.402599e+11,1.216735e+12,1.198896e+12,1.341887e+12,1.675615e+12,1.823050e+12,1.827638e+12,1.856722e+12,2.039127e+12,2.103588e+12
Japan,4,52780,52281,557023,132113,10.55,203,1.898400e+10,149,10.232820,4.601663e+12,4.579751e+12,5.106679e+12,5.289493e+12,5.759072e+12,6.233147e+12,6.272363e+12,5.212328e+12,4.896994e+12,4.444931e+12
United Kingdom,5,47141,45928,748994,132737,15.89,244,7.920000e+09,124,10.600470,2.717060e+12,3.106182e+12,2.938882e+12,2.425798e+12,2.491110e+12,2.674891e+12,2.719158e+12,2.803291e+12,3.087166e+12,2.956574e+12
Germany,6,42343,41464,528645,115432,12.48,211,1.326100e+10,165,17.901530,2.994704e+12,3.425578e+12,3.745264e+12,3.411261e+12,3.399668e+12,3.749315e+12,3.527143e+12,3.733805e+12,3.889093e+12,3.357586e+12
Russian Federation,7,39424,39189,142937,67935,3.63,96,3.070900e+10,214,17.288680,9.899305e+11,1.299706e+12,1.660846e+12,1.222644e+12,1.524917e+12,2.045926e+12,2.208296e+12,2.292473e+12,2.059242e+12,1.363481e+12
Canada,8,35588,34940,665415,113363,18.70,246,1.043100e+10,296,61.945430,1.319265e+12,1.468820e+12,1.552990e+12,1.374625e+12,1.617343e+12,1.793327e+12,1.828366e+12,1.846597e+12,1.805750e+12,1.556509e+12
Italy,9,31260,29959,433388,105641,13.86,177,6.530000e+09,109,33.667230,1.949552e+12,2.213102e+12,2.408655e+12,2.199929e+12,2.136100e+12,2.294994e+12,2.086958e+12,2.141924e+12,2.162010e+12,1.836638e+12


In [380]:
def answer_two():
    Top15 = answer_one()
    years = [str(year) for year in range(2006, 2016)]
    avgGDP =  Top15[years].apply(lambda x: sum(x)/len(x), axis=1)
    avgGDP.sort_values(ascending=False, inplace=True)
    return avgGDP
answer_two()

Country
United States         1.570403e+13
China                 6.927702e+12
Japan                 5.239642e+12
Germany               3.523342e+12
United Kingdom        2.792011e+12
France                2.691337e+12
Italy                 2.142986e+12
Brazil                1.988889e+12
Russian Federation    1.666746e+12
Canada                1.616359e+12
India                 1.602352e+12
Spain                 1.400727e+12
South Korea           1.221372e+12
Australia             1.207106e+12
Iran                  4.563261e+11
dtype: float64

In [397]:
def answer_three():
    Top15 = answer_one()
    avg = answer_two().index[5]
    diff = Top15.loc[avg]["2015"] - Top15.loc[avg]["2006"]
    return diff
answer_three()

118652421857.7998

In [ ]:
def answer_four():
    Top15 = answer_one()
    return "ANSWER"